# 11-1. 들어가며

1. 학습 목표

(1) 번역 모델이 발전해 온 과정을 살펴본다.

(2) 번역을 생성하는 여러 가지 방법을 이해한다.

(3) 주어진 데이터로 더 높은 성능을 만들어 내는 법을 배운다.

(4) 자연어 처리의 성능을 측정하기 위한 지표를 배운다.

# 11-2. 번역의 흐름

1. 규칙 기반 기계 번역

잠시 한영 번역가가 되었다고 생각해 봅시다. 나는이라는 어절을 어떻게 번역할 수 있을까요? 아마 백이면 백 I am 또는 I'm 이라고 번역을 할 겁니다. 그렇다면 너는은 어떻죠? You am이라고 번역하실 분은 설마 없겠죠? You are이나 You're로 번역하는 것이 맞습니다!

나 + 는과 너 + 는 모두 는을 포함하는데, 영문에서는 는 이 am 과 are로 나뉘는 것을 알 수 있습니다. 그러니 는 = am 이라고 정의할 수도, 는 = are 이라고 정의할 수도 없죠. 먼저 등장하는 단어가 나 인지 너 인지에 따라 분기를 나눠줘야 하는 거예요.

이처럼 번역할 때 경우의 수를 직접 정의해 주는 방식이 규칙 기반 기계 번역(RBMT, Rule-Based Machine Translation) 입니다.

이 수많은 규칙들은 모두 언어학을 기반으로 하기 때문에, 개발 과정에 언어학자가 동반되어야 했습니다.

그리고 첫 성과가 난 것은 1954년, 미국의 조지타운대와 IBM이 직접 정의한 언어 규칙을 통해 60개의 러시아어 문장을 영어로 번역하는 데에 성공합니다! 이에 대한 자세한 내용은 논문에 들어있으니 아래 링크를 참고하길 바랍니다.

[역사 속 사진 1954년 1월 7일, 기계 번역의 역사를 만든 '조지타운-IBM 실험' 시연](https://m.blog.naver.com/newheater/221763031420)

그러나 규칙 기반 기계 번역은 한계가 명확하다. 규칙에 없는 문장이 들어올 경우 번역이 불가능하고 유연성이 떨어진다. 무엇보다 모든 규칙을 정의하는 과정이 너무나 복잡하고 오랜 시간이 필요하다.


2. 통계적 기계 번역

단점이 명확한 규칙 기반 기계 번역을 개선하고자 하는 시도는 당연히 존재했으며, 아무래도 그것은 직접 구현해 본 쪽이 유리했나 봅니다. 1988년에 IBM이 Model 1을 통해 새로운 번역 방식을 선보였죠. 수많은 데이터로부터 통계적 확률을 구해 번역을 진행하는 통계적 기계 번역(SMT, Statistical Machine Translation) 이 바로 그것입니다.

[위키독스: 통계적 언어 모델](https://wikidocs.net/21687)

Q1. 아래와 각 단어에 대한 예측 확률이 주어졌을 때, 문장 "나는 커피를 마신다" 에 대한 확률을 구해보세요.

P(는|나) = 0.9, P(커피|나는) = 0.5, P(밥|나는) = 0.3, P(를|나는 커피) = 0.95, P(마신다|나는 커피를) = 0.73, P(먹는다|나는 밥을) = 0.81
- P(나는 커피를 마신다) = P(는|나) x P(커피|나는) x P(를|나는 커피) x P(마신다|나는 커피를) = 0.9 x 0.5 x 0.95 x 0.73 = 0.312075

Q2. 통계적 모델의 장점과 한계점을 하나씩만 적어봅시다.
- 장점 : 일일이 규칙을 정의해야 하는 규칙 기반 모델보다 개발에 대한 Cost가 훨씬 적다.
많은 데이터가 뒷받침될 경우, 훨씬 더 유연한 문장 생성이 가능하다(많이 사용되는 문장)

- 단점 : 본 적이 없는 문장에 대한 모델링이 불가능하다. 즉 데이터 의존적이다.
문장의 어순에 대한 고려가 없기 때문에 다소 어색한 문장이 생성될 수 있다.

Q3. 희소 문제(sparsity problem)란 무엇인가요?
- 목표하는 문장이 훈련 데이터에 없었던 시퀀스일 경우 올바른 문장임에도 확률이 0으로 정의되는 문제이다. 즉 충분한 데이터를 관측하지 못하여 언어를 정확히 모델링하지 못하는 문제이다.

![](https://d3s0tskafalll9.cloudfront.net/media/images/GN-6-L-02.max-800x600.png)
<center>통계적 기계 번역 작동 예시</center>

번역은 자연스러운 문장을 생성하는 것뿐만 아니라 번역문의 문법 구조도 고려해야 하기 때문에 단순히 P(Target | Source) 만을 고려해서는 안 된답니다. 원문과 번역문, 각 단어 간의 매핑 관계를 추가로 고려해야 하는데, 그 관계를 정렬(Alignment) 이라고 부릅니다.

정렬에는 퍼틸리티(Fertility) 와 왜곡(Distortion), 두 가지 요소가 포함됩니다.

퍼틸리티는 직역하면 '출생률'정도 되는 의미를 갖고있다. 원문의 각 단어가 번역 후에 몇 개의 단어로 나타나는지를 의미하는 값이다. 위 사진를 예를들면, Everuone의 퍼틸리티는 2이다. 그리고 to, the의 퍼틸리티는 0이다. 번역에 직접적으로 등장하지 않는다는 의미이다.

퍼틸리티에 대한 확률은 p(n|w)로 정의되며 n은 퍼틸리티 값, w는 원문의 단어이다.

music은 높은 확률로 음악으로만 번역할테니 p(1|music)은 0.9정도로 표현할 수 있다.

왜곡은 원문의 단어가 번역문에서 존재하는 위치를 나타냅니다. 위의 예에서 Clapped는 손뼉(6) 을(7) 쳤다(8)로 번력되므로 Clapped의 왜곡은 (6,7,8)로 나타난다.

왜곡에 대한 확률은 p(t|s, l) 로 정의되며 t은 번역문에서 각 단어의 위치, s는 원문에서 각 단어의 위치, 그리고 l은 번역문의 길이입니다.

Everyone(1) 으로 시작하는 문장은 모두(1) / 가(2) 라고 번역될 확률이 높으니 p(1|1, 8) x p(2|1, 8) 역시 제법 높을 것을 추측할 수 있습니다.

예시)

E: Everyone(1, 2) clapped(6, 7, 8) in(4) time(5) to(·) the(·) music(3)

-> K: 모두(1) 가(2) 음악(3) 에(4) 맞춰(5) 손뼉(6) 을(7) 쳤다(8)

수식

p(E|K) =
	{p(2|Everyone) x p(1|1, 8) x p(2|1, 8) x p(모두|Everyone) x p(가|Everyone)} x
	{ /*  수식을 작성하세요! */ } x
	{p(1|in) x p(4|3, 8) x p(에|in)} x
	{p(1|time) x p(5|4, 8) x p(맞춰|time)} x
	{p(0|to) x} x
	{p(0|the) x} x
	{p(1|music) x p(3|7, 8) x p(음악|music)}

Q4. 위 수식의 빈칸에 들어갈 식을 작성하라
- p(3|clapped) x p(6|2, 8) x p(7|2, 8) x p(8|2, 8) x p(손뼉|clapped) x p(을|clapped) x p(쳤다|clapped)







# 11-3. 지적 생성을 위한 넓고 얕은 탐색 (1) Greedy Decoding

예전 프로젝트에서 사용했던 생성 코드를 잠시 살펴보자.

In [ ]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)

    end_token = tokenizer.word_index["<end>"]

    while True:
        predict = model(test_tensor)
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]

        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

이 함수는 훈련시킨 모델이 문장을 생성할 수 있게 해주는 함수이다. 주목해야할 부분은 바로 단어를 결정하는 부분이다.

In [ ]:
predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]

모델이 예측한 값을 Softmax를 통해 확률 값으로 변환한 후, 가장 높은 확률을 갖는 단어가 다음 단어로 결정된다. 이는 탐욕 알고리즘이 사용된 것이다. 탐욕적인 방법으로 문장을 Decoding 하니 기계 번역에서는 이를 Greedy Decoding이라고 칭합니다. 아래 링크 참고

[Greedy Algorithm (탐욕 알고리즘)](https://janghw.tistory.com/entry/%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98-Greedy-Algorithm-%ED%83%90%EC%9A%95-%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98)

탐욕적인 방법은 효율작이지만 최적의 해를 구해준다는 보장이 없다. 즉 우리는 지금 최고의 번역을 생성하고 있는 것이 아니다. 아래 그림은 예시이다.

![](https://d3s0tskafalll9.cloudfront.net/media/images/GN-6-L-03.max-800x600.png)
<center>탐욕적인 방법 예시</center>

우리는 문장이 존재할 확률을 구할 수 있다. 우리는 저 영어 문장을 볼 때 커피를 한 잔 마셔도 될까요?라고 번역할 확률이 높다. 하지만 훈련 데이터가 실제 세계의 모든 데이토를 포함할 수는 없기 때문에, have이 마시다로 사용되는 경우가 훈련데이터에 적거나 없다면 탐욕적인 방법은 가지다로 번역할 수밖에 없을 것이다.


# 11-4. 지적 생성을 위한 넓고 얕은 탐색 (2) Beam Search

탐욕적인 방법을 해소하기 위하여 모든 문장을 다 만들어보는것은 O(V^L)의 복잡도를 갖는다는 것을 제외하고 좋은 방법이긴하다. 확실히 원하는 문장이 만들어질 것이라는 보장이 있기 때문이다. 효율성이 문제이니 이 부분을 개선하면 좋은 방법이지 않을까?

Beam Search는 그런 생각에서 출발한 알고리즘이다. 단어 사전으로 만들 수 있는 모든 문장을 만드는 대신, 지금 상황에서 가장 높은 확률을 갖는 Top-k 문장만 남기는 것이다.

![](https://d3s0tskafalll9.cloudfront.net/media/images/GN-6-L-04.max-800x600.png)
<center>Beam Search</center>

상위 몇 개의 문장을 기억할지는 Beam Search로 정의해 줄 수 있다. 위 예시는 Beam Size를 2로 하는 Beam Search를 표현한 것이죠. Beam Size는 연산량과 성능 간의 Trade-off 관계를 가지고 있습니다. 

다시 말해, 데이터가 많다면 Beam Size를 키우면 키울수록 성능이 좋아진다는 것이다. 대체적으로 5 or 10을 적합하다고 한다.

아래는 Beam Search 구현 코드이다.






In [1]:
# Beam Search 구현

import math
import numpy as np

def beam_search_decoder(prob, beam_size):
  sequences = [[[], 1.0]] # 생성된 문장과 점수를 저장

  for tok in prob:
    all_candidates = []

    for seq, score in sequences:
      for idx, p in enumerate(tok): # 각 단어의 확률을 총점에 누적 곱
        candidate = [seq + [idx], score * -math.log(-(p-1))]
        all_candidates.append(candidate)

    ordered = sorted(all_candidates,
                     key = lambda tup:tup[1],
                     reverse = True) # 총점 순 정렬
    sequences = ordered[:beam_size] # Beam Size에 해당하는 문장만 저장 
  
  return sequences

In [2]:
vocab = {
    0: "<pad>",
    1: "까요?",
    2: "커피",
    3: "마셔",
    4: "가져",
    5: "될",
    6: "를",
    7: "한",
    8: "잔",
    9: "도",
}

prob_seq = [[0.01, 0.01, 0.60, 0.32, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
            [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.75, 0.01, 0.01, 0.17],
            [0.01, 0.01, 0.01, 0.35, 0.48, 0.10, 0.01, 0.01, 0.01, 0.01],
            [0.24, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.68],
            [0.01, 0.01, 0.12, 0.01, 0.01, 0.80, 0.01, 0.01, 0.01, 0.01],
            [0.01, 0.81, 0.01, 0.01, 0.01, 0.01, 0.11, 0.01, 0.01, 0.01],
            [0.70, 0.22, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
            [0.91, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
            [0.91, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
            [0.91, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]]

prob_seq = np.array(prob_seq)
beam_size = 3

result = beam_search_decoder(prob_seq, beam_size)

for seq, score in result:
    sentence = ""

    for word in seq:
        sentence += vocab[word] + " "

    print(sentence, "// Score: %.4f" % score)

커피 를 가져 도 될 까요? <pad> <pad> <pad> <pad>  // Score: 42.5243
커피 를 마셔 도 될 까요? <pad> <pad> <pad> <pad>  // Score: 28.0135
마셔 를 가져 도 될 까요? <pad> <pad> <pad> <pad>  // Score: 17.8983


훈련 데이터 세계에서는 커피 를 마셔 라고도 부르나 보네요. 우리가 확인하고자 했던 커피를 마셔도 될까요? 에 더하여 마셔를 가져도 될까요? 까지 확인할 수 있었습니다. 굳이 고정된 개수의 문장을 얻지 않아도 된다면 적당한 Beam Size를 설정해 준 후 Score를 기준으로 필터링하는 방법도 좋겠죠?

유의!

- Beam Search는 사람이 직접 좋은 번역을 고를 수 있게 상위 K개의 결과를 보여줄 뿐이라서 학습에 직접적으로 적용할 수는 없습니다. 모델 입장에서는 뭐가 좋은 번역인지 알 수 없으니까요! 즉 모델 학습 단계에서 Beam Search를 사용하지는 않습니다.


# 11-5. 지적 생성을 위한 넓고 얕은 탐색 (3) Sampling

Sampling은 자주 사용되는 방법은 아니지만 흥미로운 방법이다. 앞서 살펴본 방법들은 다음의 단어로 나올 확률이 높은 단어를 선택하는 방식이었다. 이 방식은 동일안 입력에 대해 늘 동일한 결과가 만들어집니다.

지금까지는 확률을 기준으로 단어를 선택해왔으니, 확률적으로 단어를 뽑는 방법이 Sampling이다.

![](https://d3s0tskafalll9.cloudfront.net/media/images/GN-6-L-05.max-800x600.png)
<center>Sampling</center>

언어 모델은 반복적으로 다음 단어에 대한 확률 분포를 생성하기 때문에 그 확률 분포를 기반으로 랜덤하게 단어를 뽑아 보자는 거죠. 위 예시를 따라서 100문장을 만들면 이상적인 경우 55개의 내가 사랑하는 귀여운 고양이, 37개의 내가 사랑하는 귀여운 강아지... 와 같이 문장이 생성되는 겁니다. 높은 확률을 갖는 단어를 택하는 경우가 가장 많기 때문에 랜덤이지만 지나치게 뜬금없는 문장이 생성되지는 않을 것이다.

Sampling을 사용할만한 곳은 Back Translation(역번역)이 대표적인 사례이다. 또한 강화 학습의 입실론 그리디(E-Greedy)와도 궤를 함께하기 때문에 자연어 처리에 강화 학습을 적용한 경우에 등장하기도 한다.



# 11-6. 방과 후 번역 수업 (1) Data Augmentation

Data Augmentation은 '데이터 증가', '데이터 확대' 등으로 직역되며 훈련 데이터를 수십 배까지도 부풀리는 기술을 의미합니다! 주로 이미지 처리 영역에서 많이 사용되는데요, 그 방법을 이해하고 나면 '그럴 수밖에 없겠구나...' 생각이 드실 겁니다. 아래 웹페이지에서 자세한 내용을 살펴보시죠!

[Data Preprocessing & Augmentation](https://nittaku.tistory.com/m/272)

Q6. 각 모델에 사용된 특수한 기법을 제외하고 윗글에서는 총 9가지의 Augmentation 기법을 서술
- 좌우 반전 (Flipping과 중복): 이미지를 좌우로 반전시킨다.
- Crop: 이미지의 특정 부분을 잘라낸다.
- 밝기 조절: 이미지의 밝기를 변화시킨다.
- Rotation: 이미지를 0 ~ 360도 회전시킨다.
- Shifting: 이미지를 약 10px 움직인다.끝단이 잘려 나가는 효과가 있다.
- Rescaling: 이미지를 1.0 ~ 1.6배 키운다.
- Flipping: 이미지를 상하 / 좌우 반전을 한다.
- Shearing: 약 -20 ~ 20도 정도로 이미지를 찌그러트린다.
- Stretching: 이미지를 가로 / 세로 약 1.0 ~ 1.3배 늘린다.



# 11-7. 방과 후 번역 수업 (2) Lexical Substitution

NLP에서 사용가능한 Data Augmentaion은 Lexical Substitution으로 '어휘 대체'로 직역됩니다.

1. 동의어 기반 대체
- 시소러스(Thesaurus) 란, 어떤 단어의 동의어나 유의어를 집중적으로 구축해놓은 사전을 의미합니다. 동의어 기반 대체는 이 시소러스를 활용한 방법입니다. 대표적인 시소러스인 워드넷(WordNet) 을 잘 다룬 글을 첨부합니다!

[시소러스를 활용한 단어 의미 파악](https://kh-kim.gitbook.io/natural-language-processing-with-pytorch/00-cover-4/03-wordnet)

Q7. 워드넷은 트리 구조가 아닌 유향 비순환 그래프 형태로 구축되었는데요. 그 이유는 무엇인가요?
-

Q8. 한국어로 구축된 워드넷도 있나요? 있다면 각 워드넷의 이름을 적어보세요.
-

![](https://d3s0tskafalll9.cloudfront.net/media/original_images/GN-6-L-07.png)
<center></center>

![](https://d3s0tskafalll9.cloudfront.net/media/images/GN-6-L-06.max-800x600.png)
<center></center>

동의어를 기반으로 대체하는 것은 아주 좋은 방법이지만, 규칙 기반 기계 번역처럼 모든 것을 사람이 정의해야 한다는 것이 단점입니다. 

2. Embedding 활용 대체
- 동의어 기반 대체의 단점을 보완할 방법은 Pre-training Word Embedding을 활용하는 것이다. 우리는 이미 Word2Vec이나 GloVe등의 기법을 배웠다. 이 기법들을 통해 학습된 Embedding은 유사한 단어들끼리 비슷한 공간에 밀집되던 것을 기억한다. 사람이 일일이 정의한 데이터베이스 대신, 이 Embedding의 유사도를 기반으로 단어를 대체하면 훨씬 편리하다.

![](https://d3s0tskafalll9.cloudfront.net/media/images/GN-6-L-08.max-800x600.png)
<center></center>

gensim 라이브러리를 활용하면 이 방법은 아주 쉽게 사용할 수 있겠어요! 단어를 유사도 순으로 정렬해 보여주는 most_similar() 함수가 있다.

3. TF-IDF 기반 대체

![](https://d3s0tskafalll9.cloudfront.net/media/images/GN-6-L-09.max-800x600.png)
<center></center>

TF-IDF는 여러 문서를 기반으로 단어마다 중요도를 부여하는 알고리즘입니다. 문서의 핵심이 되는 소년, 피리 같은 단어들은 높은 TF-IDF 값을 부여받고, 한, 를과 같은 단어들은 낮은 TF-IDF 값을 가지게 됩니다. 이때, 낮은 TF-IDF 값을 갖는 단어들은 핵심 단어가 아니기 때문에 다른 단어로 대체해도 문맥이 크게 변하지 않는다는 것에 주목한 아이디어입니다.






# 11-8. 방과 후 번역 수업 (3) Back Translatio

Back Translatio은 단일 언어 데이터는 구하기 쉽고 병렬 쌍을 이룬 언어 데이터를 찾기는 어렵다는 문제를 해결하고자 등장하였다.

Seq2Seq의 구조를 다시 생각해보면 Source 문장을 Encoding하는 부분과 Target문장을 Decoding하는 부분을 분리하여 모듈들이 각 언어를 더 잘 처리할 수 있게 했었다. 그렇다면 Encoder에는 Source 언어로 된 문장을, Decoder에는 Target언어로 된 문장을 좀 더 훈련시켜보도록한다.

[Back Translation 정리](https://dev-sngwn.github.io/2020-01-07-back-translation/)

Q9. Synthetic Source Sentence를 생성하는 과정을 간단하게 설명해보세요.
- S(Source)-T(Target) 병렬 말뭉치가 있다고 가정하면, S->T 방향 번역 모델 A를 학습함과 동시에 T->S 방향 번역 모델 B도 동시에 학습한다. 후에 T 언어로 된 단일 언어 데이터에 모델 B를 활용해 인공적인 S 언어 데이터를 생성하고 인공적인 S 언어 데이터와 T를 병렬 쌍으로 사용한다. 이때 생성되는 인공적인 S 언어 데이터가 Synthetic Source Sentence이다.

Q10. Back Translation을 적용할 때, 문장을 생성하는 기법에 따라 성능이 천차만별입니다. 가장 좋은 성능을 보인 생성 기법은 무엇인가요?
- Beam Search에 Noise를 추가한 Beam + Noise이다.

Q11. Back Translation이 효과적인 최소 데이터 수, Sampling 기법으로 생성한 Back Translation이 효과적인 최소 데이터 수는 각각 얼마인가요?
- Back Translation: 데이터 수에 무관하게 효과적이다.
- Sampling: 64만 개 이상

 Back Translation은 현재 번역 데이터셋으로 가장 유명한 WMT2014에서 State-of-the-art 성능을 보인 알고리즘이다.


# 11-9. 방과 후 번역 수업 (4) Random Noise Injection

데이터에 포함된 적당한 노이즈는 때때로 학습에 도움이 되기도 합니다. 그런 의미에서 문장에 노이즈를 주는 것도 괜찮은 Augmentation 기법이 될 수 있다.

1. 오타 노이즈 추가
- 타자를 칠 때 주변 문자가 함께 타이핑되는 것이 자연스럽다. 이를 이용한 Augumentation 기법이 바로 오타 노이즈를 추가하는 것이다. 예를들어 올 때 아이스크림 사와 를 놀 때 아이스크림 사와 로 바꾸는 등 방법이 존재한다.

2. 공백 노이즈 추가
![](https://d3s0tskafalll9.cloudfront.net/media/images/GN-6-L-11.max-800x600.png)
<center>공백 노이즈 예시</center>
- 엄연히 말하면 완벽한 공백이 아니고, _ 토큰을 활용하며 이를 Placeholder Token(이하 공백 토큰) 이라고 부릅니다. 문장의 일부 단어를 공백 토큰으로 치환하는데요, 학습의 과적합을 방지하는 데에 좋은 효과를 볼 수 있다고 합니다.

3. 랜덤 유의어 추가
위 방식들은 노이즈 추가라고 칭했지만 대체로 문장의 의미를 유지하며 문장을 변환하는 형태였다. 이 방식은 정말 노이즈를 추가합니다. 주어진 문장에서 불용어가 아닌 단어를 랜덤하게 추출한 후, 해당 단어와 유사한 단어를 골라 문장에 삽입하는 방식이다.

![](https://d3s0tskafalll9.cloudfront.net/media/images/GN-6-L-12.max-800x600.png)
<center>랜덤 유의어 예시</center>

앞서 배운 Lexical Substitution과 비슷한 느낌이지만 원본 단어가 손실되지 않는다는 것이 조금 더 매력적이죠? Word2Vec의 아이디어를 생각하면 유사어를 삽입하는 것이 모델의 Embedding 층을 더 견고하게 만들어 줄 것이다.


참조

(1) [BLEU score 참고 1](http://incredible.ai/nlp/2020/02/29/BLEU/)

(2) [BLEU score 참고 2](https://jrc-park.tistory.com/273)



# 11-10. 채점은 어떻게?

번역 결과를 주관적으로 평가하는 것이 아니라 객관적인 지표가 필요합니다.

BLEU(Bilingual Evaluation Understudy) Score는 이에 발맞춰 등장한 번역 평가 지표입니다. '기계가 실제 번역을 얼마나 잘 재현했는가?' 를 평가하는 지표인데, '번역을 평가해야 한다!' 라는 연구자의 깊은 고민이 느껴지는 개념이니 아래 웹페이지들에서 자세한 내용을 알아보십시오.

[BLEU Score](https://donghwa-kim.github.io/BLEU.html)

Q12. 아래 번역에 대한 BLEU Score를 계산하세요.

Real: 귀여운 고양이 는 아침 식사 를 즐긴다
Pred: 고양이 고양이 고양이 고양이

- 2-gram 이상에서 매치되는 경우가 없으므로 Precision 값은 0, 따라서 BLEU는 0점
- (0.25 * 0 * 0 * 0) = 0

Q13. 아래 번역에 대한 Precision^4를 계산하세요. (Clipping 포함)

Real: 귀여운 고양이 는 아침 식사 를 즐긴다
Pred: 귀여운 고양이 는 아침 귀여운 고양이 는 아침

- 1-Gram: 4/8 = 1/2, 2-Gram: 3/7, 3-Gram: 2/6 = 1/3, 4-Gram: 1/5, Precision^4 = 1/70

아래 위키 독스 문서 참고

[위키독스 : BLEU Score](https://wikidocs.net/31695)

Q14. 결론적으로 BLEU Score는 3가지 문제점을 방지한 N-Gram 지표입니다. BLEU가 생각하는 3가지 문제점을 적고 나쁜 예를 각각 적어봅시다.

1. 같은 단어가 반복되어 높은 점수를 받는 것을 지양한다. - the the the the the the
2. 단어를 잘 재현하더라도 어순이 엉망인 번역을 지양한다. - am boy i good
3. 지나치게 짧은 번역이 높은 점수를 받는 것을 지양한다. ㅡ 브레버티 페널티 - It is

GLUE는 기계 번역 너머의 자연어 이해를 평가하기 위해 고안된 지표이다.

[GLUE 소개 자료](https://huffon.github.io/2019/11/16/glue/)




# 11-11. 실례지만, 어디 챗씨입니까? (1) 챗봇과 번역기

[챗봇 개요](https://norux.me/m/56)

Q15. 챗봇도 만드는 방법에 따라 두 가지로 나뉘는데요, 각각은 무엇인가요?
- 검색기반 모델, 생성 모델

Q16. Open Domain과 Closed Domain의 차이점에 대해 간단히 설명해보세요.
- Open Domain은 어떤 토픽이라도 오고 갈 수 있는 대화, 무한정의 주제를 다루는 소셜미디어가 대표적인 예시이다. Closed Domain은 한정된 분야에 대해서만 대화를 주고받는다. 주제를 벗어나면 답변을 생성할 수 없으며, 사용자 또한 주제 외적인 것을 기대하지 않는다.

우리는 위글에서 언급된 생성 모델에 대해서 알아본다. 번역 모델을 챗봇에 적용하는 아이디어는 Seq2Seq에서 시작한다. 앞서 역번역을 설명할 때 사용한 문단을 인용해보자

- Seq2Seq의 구조를 다시보면, Source 문장을 Encoding하는 부분과 Target 문장을 Decoding하는 부분을 분리하여 모듈들이 각 언어를 더 잘 처리할 수 있게 했었습니다. ... (하략)

챗봇의 목적은 인간과 대화이다. 질문만 하는 나라의 언어를 Source언어, 답변만 하는 나라의 언어 Target언어라고 한다면 Source 문장을 Target 문장으로 번역하는 행위는 곧 질문에 답하는 행위가 되는 셈이다. 물론 다양한 일상적인 대화도 이 같은 방법으로 학습이 가능하다.

조금 더 자세하게 알아보자면, Encoder는 Source 문장을 읽고 이해한 내용을 추상적인 고차원 문맥 벡터로 압축한다. Decoder는 Encoder가 만든 문맥 벡터를 바탕으로 Target 문장을 생성한다. 이 과정은 Source언어의 Embedding 공간 속 문장을 Target언어의 Embedding 공간으로 매핑하다고 할 수 있다.
수 많은 데이터로 학습하는 것은 그 과정을 더 잘해내기 위함이다.

![](https://d3s0tskafalll9.cloudfront.net/media/images/GN-6-L-14.max-800x600.png)

이것은 모든 Source 문장에 대해 매핑될 수 있는 고유한 Target 번역이 존재하기 때문에 가능합니다. 커피를 더 마시고 싶어 라는 문장을 영어로 올바르게 번역하는 경우의 수가 많아야 10개는 될까요? 그 모든 경우의 수 또한 I want some more coffee 에서 크게 엇나가지 않겠죠. 이처럼 사과는 무슨 색이야? 에 대한 답변도 빨간색입니다 에 수렴하기 때문에 번역기를 챗봇으로 사용하는 게 가능한 거죠!



# 11-12. 실례지만, 어디 챗씨입니까? (2) 좋은 챗봇이 되려면

좋은 챗봇을 만들기위하여 신경써야 할 대표적인 4가지가있다.

1. 200ms
- 200ms는 대화가 자연스럽게 느껴지는 답변의 공백 마지노선이다. 인간이 챗봇한테 말을 걸었을 때, 0.2초 이내에 답변이 나오는 것이 바람직하다.

2. 시공간을 담은 질문
- 고유한 답을 할 수 있는 질문이 아닌 오늘 무슨 요일이야? 처럼 매일 답변이 변하는 질문은 학습을 할 수가 없다. 특정 시곤강에 의해 결정되는 질문은 단순한 학습으로 답변할 수가 없다. 그래서 보통은 질문의 의도를 파악한 후, 시공간에 대한 질문은 다른 모듈로 연결하여 예외 처리를 하곤 한다. 왜 인공지능 비서가 10개 이상의 모듈이 필요한지에 대한 이유였다.

3. 페르소나
- 앞의 인용 글에서 인격의 일관성 (Coherent Personality) 이라는 주제로 다뤄진 것과 유사한 얘기입니다. 학습에는 주로 많은 사람들의 채팅 데이터를 모아서 사용할 수밖에 없기 때문에 모델이 대답의 일관성을 갖는다는 것은 굉장히 도전적이죠. 이때의 일관성을 모델의 인격이라고 칭하며, 그것을 페르소나라고 부릅니다.

[페르소나](https://doc.danbee.ai/blog_005_UX04_persona.html)

Q17. 챗봇의 페르소나를 설정할 때 고려해야하는 요소를 작성해주세요.
- 페르소나는 제품이나 서비스 목적에 맞게 설정되어야합니다. 목적, 개성이 나타날 수 있는 요소(성격, 목표, 동기부여, 니즈 등)

4. 대화의 일관성
- 인공지능에게 내가 한 대답을 기억하는 물어본다면 기억하지 못할 것이다. 아직까지 사용자에 관한 정보를 기억하고 이를 활용하는 모델은 없다. 대신 대화의 주제와 무관한 대화를 막아서 일관성을 유지하려는 시도는 있었다.

다음으로 소개하는 사례는 강화학습을 기반으로 한다.

[Deep Reinforcement Learning for Dialogue Generation](https://arxiv.org/pdf/1606.01541.pdf)

기존의 Source 질문을 Target 답변으로 매핑하는 훈련법은 정답을 맞히게끔 학습하기 때문에 문제가 발생합니다. 모든 질문에 대한 정답인 무슨 말인지 모르겠어요 가 존재하기 때문이죠. 또한, See you later 를 See you later 로 대답하는 것은 정답이지만, 그 대답에다 또 See you later 를 하는 것은 이상하죠? 다음에 만나자는 무한 루프에 빠지고 맙니다!

![](https://d3s0tskafalll9.cloudfront.net/media/original_images/GN-6-L-15.png)
<center>논문에 첨부된 나쁜 예</center>

재밌는 부분은 저자들이 좋은 대화를 정의하고 그걸 학습하게끔 모델을 설계했다는 거예요! 저자들은 1) 상대방이 답변하기 좋고, 2) 새로운 정보를 담고 있으며, 3) 일관성이 있는 말을 각각 보상으로 정의하여 이를 최대화하는 방향으로 학습을 진행했습니다. 딥러닝이 Loss를 최소화하듯이, 강화학습은 보상을 최대화하거든요.

토픽이 '대화의 일관성' 이니만큼 3) 일관성이 있는 말에 대해 조금만 더 살펴볼게요. 대화를 [ q1, a1, q2, a2, ... ] 라고 정의하고 q, a는 각각 질문과 답변입니다. a2를 생성할 때에는 a1과 q2를 보고 생성을 하고, 해당 문장이 생성될 확률이 곧 보상이 됩니다. 하지만 앞서 말했듯이 모르겠어요 는 모든 질문에 대해 생성될 확률이 높은 문장이므로 자칫하면 a2 가 모르겠어요 여도 큰 보상을 주게 되겠죠?

이에 저자들은 a2를 보고 q2를 유추할 수 있는지를 보상에 추가했습니다. 그렇게 되면 a2가 q2의 정보를 포함하지 않을 때 좋은 보상을 받을 수 없겠죠. 이는 곧 일관성 있는 대화를 생성하는 방향으로 학습이 진행되게끔 만듭니다!

# 11-13. 실례지만, 어디 챗씨입니까? (3) 대표적인 챗봇

1. Meena
- 구글이 만든 챗봇입니다. GPT-2보다 2배 가량 큰 모델을 9배 더 많은 데이터로 학습한 친구이다. 모델 구조는 Evolved Transformer를 사용했다. 무엇보다 자체적인 대화 평가 지표인 SSA를 제안하고 이를 분석한 부분이 인상적인 챗봇이다. 아래 페이지를 참고하여 질문에 답해보자.

[무슨 대화든 할 수 있는 에이전트를 향하여](https://brunch.co.kr/@synabreu/35)

Q18. SSA의 평가 과정에 대해 설명해보세요.
- 대화의 일관성을 위해 "Hi!" 로 대화를 시작하며, 이후 평가자는 챗봇이 생성한 발언에 대해 구체적인지와 합리적인지를 T/F로 평가한다.

2. Blender
- 메타의 Blender도 위 모델 못지않다. 모델에 페르소나를 부여하고자 하는 시도와 자체적인 평가 지표 ACUTE-Eval을 제안한 것이 인상적이다. 난이도있는 내용이므로 가볍게 읽어보도록 한다.

[논문리뷰 블렌더(Blender) - Facebook AI의 오픈 도메인 챗봇](https://littlefoxdiary.tistory.com/m/39)

Q19. 저자들은 데이터를 그냥 학습시키는 것이 아닌 모델에게 특정 부분에 대해 초점을 맞추고 학습하게끔 하는 방법을 제안했습니다. 그 방법은 무엇인가요?
- 블렌딩 스킬

Q20. 제안된 평가 지표 ACUTE-Eval에 대해 간단히 설명해보세요.
- 생성된 두 개의 긴 대화를 보고 평가자는 어떤 대화가 더 사람 같은지와 어떤 챗봇과 대화하고 싶은지를 양자택일로 평가한다.

